# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import folium

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bayanaul,50.7889,75.6956,6.95,55,16,2.15,KZ,1697202958
1,1,ushuaia,-54.8000,-68.3000,8.79,79,75,14.31,AR,1697202958
2,2,puerto natales,-51.7236,-72.4875,10.25,62,75,6.69,CL,1697202958
3,3,aripuana,-9.1667,-60.6333,33.99,42,16,0.73,BR,1697202958
4,4,lompoc,34.6391,-120.4579,18.21,100,20,3.09,US,1697202958


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
humidity_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 1000,
    frame_hight = 2000,
    size = "Humidity",
    color = "City"
)

# Display the map
humidity_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp = (21,27)
ideal_windspeed = (0, 4.5)
max_cloudiness = 0

ideal_weather = city_data_df[
    (city_data_df["Max Temp"].between(ideal_temp[0], ideal_temp[1])) &
    (city_data_df["Wind Speed"].between(ideal_windspeed[0], ideal_windspeed[1])) &
    (city_data_df['Cloudiness'].fillna(0) <= max_cloudiness)
].dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
49,49,ambodifotatra,-16.9833,49.8500,25.38,75,0,3.77,MG,1697202971
66,66,hawaiian paradise park,19.5933,-154.9731,21.69,95,0,1.54,US,1697202975
170,170,turpan,42.9333,89.1667,24.02,19,0,1.38,CN,1697202724
196,196,kailua-kona,19.6406,-155.9956,24.72,82,0,2.57,US,1697203012
209,209,empalme,27.9667,-110.8167,22.18,77,0,1.47,MX,1697203015
257,257,tura,25.5198,90.2201,22.11,80,0,1.14,IN,1697203025
268,268,sirte,31.2089,16.5887,25.24,56,0,3.47,LY,1697203028
316,316,yigo village,13.5361,144.8886,26.62,90,0,4.12,GU,1697203042
326,326,durban,-29.8579,31.0292,22.51,64,0,4.26,ZA,1697203044
333,333,benghazi,32.1167,20.0667,23.97,57,0,3.95,LY,1697203047


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
49,ambodifotatra,MG,-16.9833,49.8500,75,
66,hawaiian paradise park,US,19.5933,-154.9731,95,
170,turpan,CN,42.9333,89.1667,19,
196,kailua-kona,US,19.6406,-155.9956,82,
209,empalme,MX,27.9667,-110.8167,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000

params = {
    "categories":categories,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
     
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ambodifotatra - nearest hotel: Les Palmiers
hawaiian paradise park - nearest hotel: No hotel found
turpan - nearest hotel: Oriental
kailua-kona - nearest hotel: Kona Seaside Hotel
empalme - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
sirte - nearest hotel: فندق المدينة-سرت
yigo village - nearest hotel: No hotel found
durban - nearest hotel: Tudor
benghazi - nearest hotel: ترجاب
karystos - nearest hotel: Galaxy Hotel
vryburg - nearest hotel: International Hotel
civril - nearest hotel: Emre


,City,Country,Lat,Lng,Humidity,Hotel Name
49,ambodifotatra,MG,-16.9833,49.8500,75,Les Palmiers
66,hawaiian paradise park,US,19.5933,-154.9731,95,No hotel found
170,turpan,CN,42.9333,89.1667,19,Oriental
196,kailua-kona,US,19.6406,-155.9956,82,Kona Seaside Hotel
209,empalme,MX,27.9667,-110.8167,77,No hotel found
257,tura,IN,25.5198,90.2201,80,No hotel found
268,sirte,LY,31.2089,16.5887,56,فندق المدينة-سرت
316,yigo village,GU,13.5361,144.8886,90,No hotel found
326,durban,ZA,-29.8579,31.0292,64,Tudor
333,benghazi,LY,32.1167,20.0667,57,ترجاب


In [7]:
hotel_df.reset_index(drop=True, inplace=True)
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,ambodifotatra,MG,-16.9833,49.8500,75,Les Palmiers
1,hawaiian paradise park,US,19.5933,-154.9731,95,No hotel found
2,turpan,CN,42.9333,89.1667,19,Oriental
3,kailua-kona,US,19.6406,-155.9956,82,Kona Seaside Hotel
4,empalme,MX,27.9667,-110.8167,77,No hotel found
5,tura,IN,25.5198,90.2201,80,No hotel found
6,sirte,LY,31.2089,16.5887,56,فندق المدينة-سرت
7,yigo village,GU,13.5361,144.8886,90,No hotel found
8,durban,ZA,-29.8579,31.0292,64,Tudor
9,benghazi,LY,32.1167,20.0667,57,ترجاب


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# # Configure the map plot

hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    color="City",
    size="Humidity",
    hover_cols=["Hotel Name", "Country"]
)
# Display the map
hotel_map_plot